# Set puzzle parameters and create AoC Session

In [1]:
# set puzzle parameters
PUZZLE_DAY = 9
PUZZLE_YEAR = 2024

# import from local packages
from aoc_solver import AoCSession, AoCSolver, AoCTester
AoC_SESSION = AoCSession.from_file()

# Import additional packages

In [2]:
# import from standard library packages
from typing import Tuple

# import from third-party packages
import polars
from polars import Series
from tqdm.notebook import tqdm

# Create solver class and instance

In [3]:
class Solver(AoCSolver):
    def solve_part1(self, data: Tuple[str]) -> int:
        filesystem_length = sum(int(i) for i in data)
        filesystem = Series(['.']*filesystem_length)
        
        file_id = 0
        current_index = 0
        open_blocks = []
        for i, record in enumerate(tqdm(data)):
            current_blocks = list(range(current_index, current_index+record))
            if i % 2 == 0:
                filesystem[current_blocks] = Series([file_id]*record)
                file_id += 1
            else:
                open_blocks.extend(current_blocks)
            current_index += record
        
        open_blocks_index = 0
        for i in tqdm(range(filesystem_length-1, -1, -1)):
            if filesystem[i] =='.':
                continue
            if i <= open_blocks[open_blocks_index]:
                break
            filesystem[open_blocks[open_blocks_index]] = filesystem[i]
            filesystem[i] = '.'
            open_blocks_index += 1
        return sum(i*int(x) for i,x in enumerate(filesystem) if x != '.')

    def solve_part2(self, data: ...) -> int:
        return ...

In [4]:
solver = Solver(PUZZLE_YEAR, PUZZLE_DAY, AoC_SESSION)

# Build part 1 test case(s)

In [6]:
puzzle_instructions = solver.puzzle_instructions

part1_test_input = solver.get_value_after('For example:').create_tuple(dtype=int ,separator='')
print(f'{part1_test_input=}\n')

part1_test_output = solver.get_value_after('is the sum of these, ').as_int
print(f'{part1_test_output=}\n')

instructions_matching_text='2333133121414131402'

part1_test_input=(2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2)

instructions_matching_text='1928'

part1_test_output=1928



In [7]:
part_1_tester = AoCTester()
part_1_tester.add_test_case(part1_test_input, part1_test_output)

In [8]:
%%time
part_1_tester.run_tests(solver.solve_part1)

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

data[:5]=(2, 3, 3, 3, 1)
expected_output=1928
actual_output=1928
outputs_equal=True

CPU times: user 45.1 ms, sys: 51 μs, total: 45.2 ms
Wall time: 42.6 ms


# Determine part 1 solution

In [9]:
%%time
puzzle_input = solver.puzzle_input.create_tuple(dtype=int ,separator='')
part1_solution = solver.solve_part1(puzzle_input)
print(f'{part1_solution=}\n')

  0%|          | 0/19999 [00:00<?, ?it/s]

  0%|          | 0/95195 [00:00<?, ?it/s]

part1_solution=6401092019345

CPU times: user 1min 48s, sys: 127 ms, total: 1min 48s
Wall time: 1min 48s


# Add part 1 solution to part 1 test cases

In [ ]:
part_1_tester.add_test_case(puzzle_input, part1_solution)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Build part 2 test case(s)

In [ ]:
solver.download_instructions(overwrite=True)

In [ ]:
# part2_test_input = solver.get_value_after('...').as_int
# part2_test_input = solver.get_value_after('...').create_polars(dtype,separator)
# part2_test_input = solver.get_value_after('...').as_string
# part2_test_input = solver.get_value_after('...').create_tuple(dtype,separator)
part2_test_input = part1_test_input
print(f'{part2_test_input=}\n')

part2_test_output = solver.get_value_after('...').as_int
print(f'{part2_test_output=}\n')

In [ ]:
part_2_tester = AoCTester()
part_2_tester.add_test_case(part2_test_input, part2_test_output)

In [ ]:
%%time
part_2_tester.run_tests(solver.solve_part2)

# Determine part 2 solution

In [ ]:
%%time
part2_solution = solver.solve_part2(puzzle_input)
print(f'{part2_solution=}\n')